In [1]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
from scipy.signal import butter,filtfilt
import numpy as np
import time


Fs = 30000
signal = np.random.rand(16,120*Fs)
recording = se.NumpyRecordingExtractor(signal,Fs)

def filterRecording(recording, sampling_freq, lp_freq=300,hp_freq=6000,order=3):
    fn = sampling_freq / 2.
    band = np.array([lp_freq, hp_freq]) / fn

    b, a = butter(order, band, btype='bandpass')

    if not (np.all(np.abs(np.roots(a)) < 1) and np.all(np.abs(np.roots(a)) < 1)):
        raise ValueError('Filter is not stable')
    
    for i in range(recording._timeseries.shape[0]):
        recording._timeseries[i,:] = filtfilt(b,a,recording._timeseries[i,:])

    return recording

17:06:12 [I] klustakwik KlustaKwik2 version 0.2.6


In [2]:
%load_ext line_profiler

In [4]:
%lprun -f st.preprocessing.BandpassFilterRecording._do_filter st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000,cache=True, type='butter',chunk_size=1000)


Timer unit: 1e-06 s

Total time: 12.8504 s
File: /Users/teristam/anaconda3/lib/python3.7/site-packages/spiketoolkit/preprocessing/bandpass_filter.py
Function: _do_filter at line 73

Line #      Hits         Time  Per Hit   % Time  Line Contents
    73                                               def _do_filter(self, chunk):
    74      3600      12920.0      3.6      0.1          sampling_frequency = self._recording.get_sampling_frequency()
    75      3600       5882.0      1.6      0.0          M = chunk.shape[0]
    76      3600       1982.0      0.6      0.0          chunk2 = chunk
    77                                                   # Do the actual filtering with a DFT with real input
    78      3600       3650.0      1.0      0.0          if self._type == 'fft':
    79                                                       chunk_fft = np.fft.rfft(chunk2)
    80                                                       kernel = self._create_filter_kernel(
    81                  

In [6]:
%lprun -f st.preprocessing.filterrecording.FilterRecording.get_traces st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000,cache=True, type='butter',chunk_size=1000)


Timer unit: 1e-06 s

Total time: 14.8611 s
File: /Users/teristam/anaconda3/lib/python3.7/site-packages/spiketoolkit/preprocessing/filterrecording.py
Function: get_traces at line 28

Line #      Hits         Time  Per Hit   % Time  Line Contents
    28                                               def get_traces(self, channel_ids=None, start_frame=None, end_frame=None):
    29         1          1.0      1.0      0.0          if start_frame is None:
    30         1          1.0      1.0      0.0              start_frame = 0
    31         1          0.0      0.0      0.0          if end_frame is None:
    32         1          5.0      5.0      0.0              end_frame = self.get_num_frames()
    33         1          1.0      1.0      0.0          if channel_ids is None:
    34         1          3.0      3.0      0.0              channel_ids = self.get_channel_ids()
    35         1          1.0      1.0      0.0          if self._cache and self._traces is not None:
    36         

In [7]:
%lprun -f st.preprocessing.filterrecording.FilterRecording._get_filtered_chunk st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000,cache=True, type='butter',chunk_size=1000)


Timer unit: 1e-06 s

Total time: 15.9764 s
File: /Users/teristam/anaconda3/lib/python3.7/site-packages/spiketoolkit/preprocessing/filterrecording.py
Function: _get_filtered_chunk at line 59

Line #      Hits         Time  Per Hit   % Time  Line Contents
    59                                               def _get_filtered_chunk(self, ind):
    60      3600      10384.0      2.9      0.1          code = str(ind)
    61      3600       8958.0      2.5      0.1          chunk0 = self._filtered_chunk_cache.get(code)
    62      3600       1882.0      0.5      0.0          if chunk0 is not None:
    63                                                       return chunk0
    64                                                   else:
    65      3600       3034.0      0.8      0.0              start0 = ind * self._chunk_size
    66      3600       2661.0      0.7      0.0              end0 = (ind + 1) * self._chunk_size
    67      3600   15917077.0   4421.4     99.6              chunk1 = sel

In [8]:
%lprun -f st.preprocessing.BandpassFilterRecording.filter_chunk st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000,cache=True, type='butter',chunk_size=1000)


Timer unit: 1e-06 s

Total time: 14.7475 s
File: /Users/teristam/anaconda3/lib/python3.7/site-packages/spiketoolkit/preprocessing/bandpass_filter.py
Function: filter_chunk at line 43

Line #      Hits         Time  Per Hit   % Time  Line Contents
    43                                               def filter_chunk(self, *, start_frame, end_frame):
    44      3600       3219.0      0.9      0.0          padding = 3000
    45      3600       2026.0      0.6      0.0          i1 = start_frame - padding
    46      3600       1479.0      0.4      0.0          i2 = end_frame + padding
    47      3600    2270062.0    630.6     15.4          padded_chunk = self._read_chunk(i1, i2)
    48      3600   12460913.0   3461.4     84.5          filtered_padded_chunk = self._do_filter(padded_chunk)
    49      3600       9772.0      2.7      0.1          return filtered_padded_chunk[:, start_frame - i1:end_frame - i1]

In [1]:
import numpy as np

In [2]:
a = np.random.randint(0,10,10)

In [3]:
a

array([7, 4, 4, 8, 6, 2, 2, 8, 7, 1])

In [4]:
d=np.diff(a)

In [5]:
d

array([-3,  0,  4, -2, -4,  0,  6, -1, -6])

In [6]:
np.where((d>=0) & (d<=5))

(array([1, 2, 5]),)